In [4]:
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import os
from dotenv import load_dotenv
load_dotenv()
import utils
%load_ext autoreload
%autoreload 2

In [3]:
model = AzureChatOpenAI(
    deployment_name= "gpt-4o",
    api_key=os.getenv('AZURE_OPENAI_API_KEY'),
    api_version="2023-09-15-preview",
    azure_endpoint="https://acsstscdamoai02.openai.azure.com/"
)

C:\Users\monojit.mandal\AppData\Local\Temp\ipykernel_22168\2458642598.py:1: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  model = AzureChatOpenAI(


In [30]:
contract1 = utils.ContractOffer(
    price_per_unit = 95,
    quantity = 10000,
    bundling_unit = None,
    bundling_amount = 20000,
    bundling_discount = 5,
    payment_term_offer = utils.PaymentTerm.NET60,
    payment_term_markup = 3,
    delivery_timeline = 10,
    contract_period = 2,
    contract_inflation_by_year = [0,3],
    rebates_threshold_unit = 12000,
    rebates_discount = 5,
    warranty = 1,
    incoterms = utils.Incoterms.DDP
)
contract2 = utils.ContractOffer(
    price_per_unit = 100,
    quantity = 10000,
    bundling_unit = None,
    bundling_amount = None,
    bundling_discount = None,
    payment_term=utils.PaymentTerm.NET10,
    delivery_timeline = 7,
    contract_period = 1,
    contract_inflation_by_year = [0],
    rebates_threshold_unit = 11000,
    rebates_discount = 1,
    warranty = 1,
    incoterms = utils.Incoterms.FOB
)
actual = utils.ContractActual(
    quantity = 10000,
    # bundling_ind = True,
    # payment_term = utils.PaymentTerm.NET60
)
actual.calculate_TCO_yearly(offer=contract2)

Cost after bundling:  1000000
Cost after bundling:  1000000
Cost after bundling and payment term 998356.1643835617
Cost after rebate 998356.1643835617


998356.1643835617

In [1]:
# contract1 = ContractOffer(
#     price_per_unit = 95,
#     quantity = 10000,
#     bundling_unit = None,
#     bundling_amount = 20000,
#     bundling_discount = 5,
#     payment_term_offer = PaymentTerm.NET60,
#     payment_term_markup = 3,
#     delivery_timeline = 10,
#     contract_period = 2,
#     contract_inflation_by_year = [0,3],
#     rebates_threshold_unit = 12000,
#     rebates_discount = 5,
#     warranty = 1,
#     incoterms = Incoterms.DDP
# )
# def calculate_final_price(offer:ContractOffer,actual:ContractActual):
#     if actual.quantity == None:
#         quantity = offer.quantity
#     else:
#         quantity = actual.quantity
#     cost = offer.price_per_unit*quantity
#     cost_bundling = (cost + offer.bundling_amount)*(1+offer.bundling_discount/100)
#     if cost_bundling <= cost:
#         bundling_ind = True
#         tco = cost_bundling
#     else:
#         bundling_ind = False
#         tco = cost_bundling
#     if offer.payment_term.value == PaymentTerm.NET60.value:
#         cost_pt = tco*(1+offer.payment_term_markup/100) - tco*(6/100)*(30/365)
#     if cost_pt <= tco:
#         markup_ind = True
#         tco = cost_pt
#     else:
#         markup_ind = False
#     tco_by_year = []
#     for inflation in offer.contract_inflation_by_year:
#         tco = tco*(1 + inflation/100)
#         tco_by_year.append(tco)
#     tco = sum(tco_by_year)/len(tco_by_year)
#     return tco

In [ ]:
# Inputs
price_per_unit = 95
quantity = 10000
bundling_unit = None
bundling_amount = 20000
bundling_discount = 5
payment_term = 'NET30'
payment_term_offer = 'NET60'
payment_term_markup = 3
delivery_timeline = 10
contract_period = 2
inflation_by_year = [0,3]
rebates_threshold_unit = 12000
rebates_discount = 5
warranty = 1
incoterms = 'DDP'

# TCO Calculation
price = price_per_unit*quantity
price_with_bundling = (price + bundling_amount)*(1-bundling_discount/100)
if price_with_bundling <= price:
    bundling_ind = True
final_price = min(price_with_bundling,price)
price_with_payment_term = final_price*(1+payment_term_markup/100) - final_price*(6/100)*30/365
if price_with_payment_term <= price:
    markup_ind = True
final_price = min(price_with_payment_term,final_price)
price_by_year = []
for inflation in inflation_by_year:
    final_price = final_price*(1 + inflation/100)
    price_by_year.append(final_price)
final_price = sum(price_by_year)/len(inflation_by_year)
print(final_price)

935322.5


In [11]:
len(inflation_by_year)

2

In [4]:
from langchain_core.messages import SystemMessage, AIMessage,trim_messages,HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


start_conversation = '''
You are a negotiator of the company. 
Your job is to negotiate with Suppliers to your company.
Please be polite & keep your tone as experienced professional.
In each conversation you will generate an offer the format given below:

Price per unit: in $
Quantity: in units
Bundling: YES or NO
Payment Terms: NET30 or NET60 depending on input you will get
Delivery Timelines: in days
Contract Period Length: in years.
Rebates: YES or NO
Warranties: in years
Incoterms: DDP or FOB

Your starting offer is

Price per unit: 100 $
Quantity: 1000 units
Bundling: YES
Payment Terms: NET60
Delivery Timelines: 10 days
Contract Period Length: 2 years.
Rebates: YES
Warranties: 1 years
Incoterms: DDP
'''

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            start_conversation,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

messages = [
    AIMessage(
        content='''
        Hi, I would like to renew our previous contract with you. 
        Please check the offer and let us know if you agree to renew it

        Price per unit: 100 $
        Quantity: 1000 units
        Bundling: YES
        Payment Terms: NET60
        Delivery Timelines: 10 days
        Contract Period Length: 2 years.
        Rebates: YES
        Warranties: 1 years
        Incoterms: DDP
        '''
    ),
    HumanMessage(content = 'Hi, I looked at the offer It seems low')
]



chain = prompt | model

trimmer = trim_messages(
    max_tokens=1000,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

trimmed_messages = trimmer.invoke(messages)
response = chain.invoke(
    {"messages": trimmed_messages, "language": 'English'}
)
response

AIMessage(content='Hello,\n\nThank you for reaching out and for considering our offer. We are committed to establishing a fair and mutually beneficial agreement with our suppliers. While we have proposed an initial offer, we are open to discussing and negotiating terms to better align with your expectations and requirements.\n\nCould you please let me know which specific aspects of the offer you feel could be improved or adjusted? This will help us in tailoring a proposal that is satisfactory for both parties.\n\nLooking forward to your feedback.\n\nBest regards.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 97, 'prompt_tokens': 205, 'total_tokens': 302, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_04751d0b65', 'finish_reason': 'stop', 'logprobs': None}, id='run-3b1f3f81-9d35-49ca-8c71-074c1249e724-0')